# Data preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
CITY_NAME = 'LANCASTER'
STORE_NAME = 'LANCASTER FSC'

## Sales/traffic Data

### Merge (if not done before)

In [19]:
sales_clean = pd.read_csv('data/1208_Lancaster_sales_cleaned.csv')

In [ ]:
# sales_clean.head(3)

In [21]:
sales_clean.date = pd.to_datetime(sales_clean.date, format='%Y-%m-%d')
sales_clean = sales_clean[['date', 'sales_original', 'sales_cleaned']]

In [22]:
traffic_clean = pd.read_csv('data/1208_Lancaster_traffic_cleaned.csv')

In [ ]:
# traffic_clean.head(3)

In [23]:
traffic_clean.date = pd.to_datetime(traffic_clean.date, format='%Y-%m-%d')
traffic_clean = traffic_clean[['date', 'traffic_original', 'traffic_cleaned']]

In [24]:
assert sales_clean.shape[0] == traffic_clean.shape[0]

In [25]:
sales_traffic_clean = pd.merge(sales_clean, traffic_clean, on='date')

In [27]:
sales_traffic_clean.to_csv('sales_traffic_' + STORE_NAME + '.csv', index=False)

### Read data (if merged before)

In [28]:
sales_traffic_clean = pd.read_csv('sales_traffic_' + STORE_NAME + '.csv')

In [29]:
sales_traffic_clean.head(3)

,date,sales_original,sales_cleaned,traffic_original,traffic_cleaned
0,2017-08-25,0.0,-16421.263373,0,-6489.118281
1,2017-08-26,0.0,-29612.355292,265,-11523.259562
2,2017-08-27,0.0,-17170.788042,2055,-5957.016460


In [31]:
sales_traffic_clean.date = pd.to_datetime(sales_traffic_clean.date, format='%Y-%m-%d')

In [32]:
# Constant date values
DATE_MIN = sales_traffic_clean.date.min().normalize()
DATE_MAX = sales_traffic_clean.date.max().normalize()

In [33]:
DATE_MIN, DATE_MAX

(Timestamp('2017-08-25 00:00:00'), Timestamp('2019-09-28 00:00:00'))

## Events Data

In [35]:
# Constant location for stores
LAT_STORE, LON_STORE = {}, {}

LAT_STORE['ORLANDO FOA'] = 28.473595
LAT_STORE['LAKE BUENA VISTA FOA'] = 28.387852
LAT_STORE['LANCASTER FSC'] = 40.025636
LAT_STORE['LAS VEGAS NORTH'] = 36.170727
LAT_STORE['LAS VEGAS SOUTH'] = 36.056725

LON_STORE['ORLANDO FOA'] = -81.451615
LON_STORE['LAKE BUENA VISTA FOA'] = -81.493674
LON_STORE['LANCASTER FSC'] = -76.217167
LON_STORE['LAS VEGAS NORTH'] = -115.157651
LON_STORE['LAS VEGAS SOUTH'] = -115.170121

In [37]:
events = pd.read_csv('events_' + CITY_NAME + '.csv')

In [38]:
events.head(5)

,id,title,description,start,end,predicted_end,duration,labels,category,timezone,...,location,venue_name,venue_formatted_address,scope,rank,local_rank,aviation_rank,state,first_seen,venue_type
0,lLvaYJOkkzk5,The Sweet Sorrows in Concert at Emmaus Road Cafe,"""Sammy and Kylie Horner, collectively known as...",2016-10-08T23:00:00Z,2016-10-09T02:00:00Z,NaN,10800,"""concert,music""",concerts,America/New_York,...,"""40.033773,-76.250177""",Emmaus Road Cafe,"1886 Lincoln Highway East\r\nLancaster, PA 176...",locality,22,48.0,0.0,active,2016-10-05T03:27:43Z,indoor
1,n25dJ6BdMdYR,Rick Kilby-One man Band!,"""Rick Kilby knows how to have fun! Always a BI...",2016-10-08T23:00:00Z,2016-10-09T02:00:00Z,NaN,10800,"""food,performing-arts""",performing-arts,America/New_York,...,"""40.033267,-76.498226""",Prudhommes' Lost Cajun Kitchen,"50 Lancaster Avenue\r\nColumbia, PA 17512\r\nU...",locality,22,43.0,NaN,active,2016-06-11T00:52:32Z,indoor
2,030e6833f057d76d21,Leif Erikson Day,"""Leif Erikson Day honors the first Scandinavia...",2016-10-09T00:00:00Z,2016-10-09T23:59:59Z,NaN,86399,"""holiday,observance""",observances,NaN,...,"""37.09024,-95.712891""",Leif Erikson Day,NaN,country,50,NaN,0.0,active,2015-01-06T12:35:08Z,outdoor
3,84bd4640e56317fad7,World Post Day,"""World Post Day marks the anniversary of the U...",2016-10-09T00:00:00Z,2016-10-09T23:59:59Z,NaN,86399,"""holiday,observance,observance-united-nations""",observances,NaN,...,"""37.09024,-95.712891""",World Post Day,NaN,country,50,NaN,0.0,active,2015-01-06T12:35:03Z,outdoor
4,paYvrEAaMVqn,J U FROZENWATER,NaN,2016-10-09T01:00:00Z,2016-10-09T01:00:00Z,NaN,0,"""concert,music""",concerts,America/New_York,...,"""40.041523,-76.309029""",Chameleon Club,"223 N Water St\r\nLancaster, PA 17603\r\nUnite...",locality,50,63.0,NaN,active,2016-09-30T00:29:19Z,indoor


In [39]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5093 entries, 0 to 5092
Data columns (total 21 columns):
id                         5093 non-null object
title                      5093 non-null object
description                2517 non-null object
start                      5093 non-null object
end                        5093 non-null object
predicted_end              139 non-null object
duration                   5093 non-null int64
labels                     5093 non-null object
category                   5093 non-null object
timezone                   4123 non-null object
country                    5093 non-null object
location                   5093 non-null object
venue_name                 4610 non-null object
venue_formatted_address    3664 non-null object
scope                      5093 non-null object
rank                       5093 non-null int64
local_rank                 3665 non-null float64
aviation_rank              3124 non-null float64
state                      509

In [40]:
events.category.unique()

array(['concerts', 'performing-arts', 'observances', 'public-holidays',
       'severe-weather', 'community', 'expos', 'conferences',
       'airport-delays', 'festivals', 'daylight-savings', 'politics',
       'disasters', 'school-holidays', 'sports'], dtype=object)

In [41]:
events.timezone.unique()

array(['America/New_York', nan, 'America/Chicago'], dtype=object)

In [42]:
import mpu

def draw_events_data(events, store_name, date_min=DATE_MIN, date_max=DATE_MAX, 
                     lat_store=LAT_STORE, lon_store=LON_STORE):
    
    def compute_time(df):
        if df['start'].hour >= 12:
            return 'pm'
        else:
            return 'am'
    
    def compute_dist(df, store_name):
        return mpu.haversine_distance((df.latitude, df.longitude), 
                                      (lat_store[store_name], lon_store[store_name]))
    
    def compute_capacity(df):
        # exp((rank + 19.769) / 9.6442) by reverse engineering
        return np.exp((df['rank'] + 19.769) / 9.6442)
    
    def compute_impact(df, method):
        if method == 'linear':
            return df['rank'] / df['distance']
        if method == 'exp':
            return np.sqrt(np.exp(df['rank'])) / np.square(df['distance'])
    
    # Split location variable into longitude and latitude
    lat = [float(events.location[i].strip("\"").split(',')[0]) \
           for i in range(len(events))]
    lon = [float(events.location[i].strip("\"").split(',')[1]) \
           for i in range(len(events))]
    events['longitude'] = lon
    events['latitude'] = lat
    
    events.start = pd.to_datetime(events.start)
    events.end = pd.to_datetime(events.end)
    
    # Extract time (am/pm)
    events['start_time'] = events.apply(compute_time, axis=1)
    
    # Generate a date range for the events
    events['start'] = events.start.dt.tz_localize(None).dt.normalize()
    events['end'] = events.end.dt.tz_localize(None).dt.normalize()
    
    date_range = events.apply(lambda x: pd.date_range(x.start, x.end).tolist(), 
                              axis=1)
    
    events = events.drop(['id', 'start', 'end', 'predicted_end', \
                          'timezone', 'country', 'location', \
                          'venue_formatted_address', 'state', 'first_seen'], 1)
    
    events_by_date = pd.DataFrame(columns = events.columns)
    dates_rearr = []
    for i in range(len(events)):
        for dates in date_range[i]:
            if (dates <= date_max) & (dates >= date_min):
                dates_rearr.append(dates)
                events_by_date = events_by_date.append(events.iloc[i])
    
    # Aggregate new features
    # Distance in miles
    events_by_date['distance'] = events_by_date.apply(compute_dist, 
                                                  store_name=store_name, axis=1).div(1.609)
    # Estimated capacity
    events_by_date['est_capacity'] = events_by_date.apply(compute_capacity, axis=1)
    # Impact
    events_by_date['impact_linear'] = events_by_date.apply(compute_impact, method='linear', axis=1)
    events_by_date['impact_exp'] = events_by_date.apply(compute_impact, method='exp', axis=1)
    
    events_by_date['date'] = dates_rearr
    # Check if an event is annual
    events_by_date['year'] = events_by_date.date.dt.year
    is_annual = events_by_date.groupby('title')['year'].nunique().to_frame('is_annual').reset_index()
    is_annual['is_annual'] = is_annual.eval('is_annual == 3').astype(int)
    events_by_date = pd.merge(events_by_date, is_annual, on='title')
    
    # Reorganize the columns
    cols = ['title', 'description', 'labels', 'category', \
            'date', 'year', 'is_annual', 'start_time', 'duration', \
            'venue_name', 'scope', 'venue_type', 'est_capacity', 'distance', 'longitude', 'latitude', \
            'rank', 'local_rank', 'aviation_rank', 'impact_linear', 'impact_exp']
    events_by_date = events_by_date[cols]
    
    # For efficiency, write out the data
    events_by_date.to_csv('events_' + store_name + '.csv', index=False)

In [43]:
draw_events_data(events, STORE_NAME)

In [44]:
events_by_date = pd.read_csv('events_' + STORE_NAME + '.csv')

In [45]:
events_by_date.date = pd.to_datetime(events_by_date.date, format='%Y-%m-%d')

In [46]:
events_by_date.head(3)

,title,description,labels,category,date,year,is_annual,start_time,duration,venue_name,...,venue_type,est_capacity,distance,longitude,latitude,rank,local_rank,aviation_rank,impact_linear,impact_exp
0,The Women's Business Center at ASSETS #SheOwns...,"""Join the Women’s Business Center at ASSETS as...","""business,conference""",conferences,2017-08-25,2017,0,pm,10800,The Candy Factory,...,indoor,214.409880,4.934731,-76.307366,40.043784,32,45.0,0.0,6.484649,3.649091e+05
1,HAZMAT - North-America - USA,"""Seven people were injured in a hazardous mate...","""disaster,hazmat""",disasters,2017-08-25,2017,1,pm,0,NaN,...,NaN,1386.165056,11.841313,-76.382741,40.141008,50,NaN,NaN,4.222505,5.135259e+08
2,HAZMAT - North-America - USA,"""One person was injured and a school sheltered...","""disaster,hazmat""",disasters,2018-10-16,2018,1,am,0,NaN,...,NaN,11026.773414,11.236597,-76.172368,40.184579,70,NaN,NaN,6.229644,1.256138e+13


In [47]:
events_by_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4305 entries, 0 to 4304
Data columns (total 21 columns):
title            4305 non-null object
description      2030 non-null object
labels           4305 non-null object
category         4305 non-null object
date             4305 non-null datetime64[ns]
year             4305 non-null int64
is_annual        4305 non-null int64
start_time       4305 non-null object
duration         4305 non-null int64
venue_name       3603 non-null object
scope            4305 non-null object
venue_type       3603 non-null object
est_capacity     4305 non-null float64
distance         4305 non-null float64
longitude        4305 non-null float64
latitude         4305 non-null float64
rank             4305 non-null int64
local_rank       2914 non-null float64
aviation_rank    2661 non-null float64
impact_linear    4305 non-null float64
impact_exp       4305 non-null float64
dtypes: datetime64[ns](1), float64(8), int64(4), object(8)
memory usage: 706.4+ KB
